# Исследование популярности компьютерных игр

Из открытых источников доступны исторические данные о продажах игр, оценки пользователей и экспертов, жанры и платформы (например, Xbox или PlayStation). Нужно выявить определяющие успешность игры закономерности. Это позволит сделать ставку на потенциально популярный продукт и спланировать рекламные кампании.

### План работы:

1. [Подготовка данных](#1.-Подготовка-данных.)
   
2. [Исследовательский анализ данных](#2.-Исследовательский-анализ-данных.)

3. [Портрет пользователя каждого региона](#3.-Портрет-пользователя-каждого-региона.)

4. [Проверка гипотез](#4.-Проверка-гипотез.)
    
5. [Общий вывод](#5.-Общий-вывод.)
___


### Описание данных
`Name` — название игры  
`Platform` — платформa  
`Year_of_Release` — год выпуска  
`Genre` — жанр игры  
`NA_sales` — продажи в Северной Америке (миллионы проданных копий)  
`EU_sales` — продажи в Европе (миллионы проданных копий)  
`JP_sales`— продажи в Японии (миллионы проданных копий)  
`Other_sales` — продажи в других странах (миллионы проданных копий)  
`Critic_Score` — оценка критиков (максимум 100)  
`User_Score` — оценка пользователей (максимум 10)  
`Rating` — рейтинг от организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию.  

---


### 1. Подготовка данных. 

In [167]:
import pandas as pd
import math
import matplotlib.pyplot as plt 
import plotly.express as px
import numpy as np
from scipy import stats as st

In [168]:
games = pd.read_csv('/datasets/games.csv') 

In [169]:
games.head(10)

,Name,Platform,Year_of_Release,Genre,NA_sales,EU_sales,JP_sales,Other_sales,Critic_Score,User_Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN
5,Tetris,GB,1989.0,Puzzle,23.20,2.26,4.22,0.58,NaN,NaN,NaN
6,New Super Mario Bros.,DS,2006.0,Platform,11.28,9.14,6.50,2.88,89.0,8.5,E
7,Wii Play,Wii,2006.0,Misc,13.96,9.18,2.93,2.84,58.0,6.6,E
8,New Super Mario Bros. Wii,Wii,2009.0,Platform,14.44,6.94,4.70,2.24,87.0,8.4,E
9,Duck Hunt,NES,1984.0,Shooter,26.93,0.63,0.28,0.47,NaN,NaN,NaN


Приведем названия столбцов к нижнему регистру для удобства:

In [170]:
games.columns = games.columns.str.lower()
games.columns

Index(['name', 'platform', 'year_of_release', 'genre', 'na_sales', 'eu_sales',
       'jp_sales', 'other_sales', 'critic_score', 'user_score', 'rating'],
      dtype='object')

In [171]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
name               16713 non-null object
platform           16715 non-null object
year_of_release    16446 non-null float64
genre              16713 non-null object
na_sales           16715 non-null float64
eu_sales           16715 non-null float64
jp_sales           16715 non-null float64
other_sales        16715 non-null float64
critic_score       8137 non-null float64
user_score         10014 non-null object
rating             9949 non-null object
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


In [172]:
games.critic_score.describe()

count    8137.000000
mean       68.967679
std        13.938165
min        13.000000
25%        60.000000
50%        71.000000
75%        79.000000
max        98.000000
Name: critic_score, dtype: float64

In [173]:
#количество пропусков
total = games.isnull().sum().sort_values(ascending=False)
percent = ((games.isnull().sum()/games.isnull().count())*100).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.style.format('{:.1f}', subset='Percent')

,Total,Percent
critic_score,8578,51.3
rating,6766,40.5
user_score,6701,40.1
year_of_release,269,1.6
genre,2,0.0
name,2,0.0
other_sales,0,0.0
jp_sales,0,0.0
eu_sales,0,0.0
na_sales,0,0.0


In [174]:
display(len(games['name'].unique()))
games[games['name'].isnull()]

11560

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
659,NaN,GEN,1993.0,NaN,1.78,0.53,0.00,0.08,NaN,NaN,NaN
14244,NaN,GEN,1993.0,NaN,0.00,0.00,0.03,0.00,NaN,NaN,NaN


В столбце `name` всего 2 пропущенных значения и они совпадают с пропусками в столбце `genre`. Можем безболезненно от них избавиться, так как в этих строках отсутствуют и все рейтинги и оценки:

In [175]:
games.drop([659, 14244], inplace=True)

Проверим нет ли ошибок или повторов в названиях жанров:

In [176]:
games['genre'] = games['genre'].str.lower()
games['genre'].unique()

array(['sports', 'platform', 'racing', 'role-playing', 'puzzle', 'misc',
       'shooter', 'simulation', 'action', 'fighting', 'adventure',
       'strategy'], dtype=object)

Проверим нет ли ошибок или повторов в рейтингах ESRB:

In [177]:
games['rating'].unique()

array(['E', nan, 'M', 'T', 'E10+', 'K-A', 'AO', 'EC', 'RP'], dtype=object)

In [178]:
games.loc[games['rating']=="K-A", 'rating']= 'E'

Рейтинг 'K-A' был заменен на 'E' после 1998, сделали такую же замену в нашей таблице.   

Проверим, есть ли дубликаты:

In [179]:
print('Полных дубликатов: ',games.duplicated().sum())

# в графе 'name' могут быть ошибки, одни и те же буквы в разном регистре и т.п., 
# поэтому проверим на дубликаты еще и не учитывая этот столбец:
print('Дубликатов по всем столбцам, кроме "name": ', games.duplicated(subset=games.columns[games.columns != 'name'],keep=False).sum())

print('Critic score ',games[games.duplicated(subset=games.columns[games.columns != 'name'],keep=False)]['critic_score'].unique())
print('User score ',games[games.duplicated(subset=games.columns[games.columns != 'name'],keep=False)]['user_score'].unique())



Полных дубликатов:  0
Дубликатов по всем столбцам, кроме "name":  1568
Critic score  [nan]
User score  [nan 'tbd']


Выявленные дубликаты имеют пропуски в `critic_score` и `user_score`, если бы в этих столбцах были бы идентичные значения, мы бы признали их дубликатами. 

In [180]:
games['year_of_release'].describe()

count    16444.000000
mean      2006.486256
std          5.875525
min       1980.000000
25%       2003.000000
50%       2007.000000
75%       2010.000000
max       2016.000000
Name: year_of_release, dtype: float64

Год выпуска `year_of_release` из формата с плавающей точкой приведем к формату *int64* так как это всегда целое число; `user_score` из строки - к *float64* с помощью *to_numeric*, т.к. это числовое значение. В столбце `user_score` есть значения 'tbd' ('to be determined'), что означает, что у игры еще недостаточно оценок для определения средней, заменим эти значения на NaN с помощью параметра errors='coerce'. `critic_score` из *float64* приведем к *int64*, так как это целые числа:

In [181]:
display(len(games.query('user_score=="tbd"')))
games['year_of_release'] = games['year_of_release'].astype('Int64')
games['critic_score'] = games['critic_score'].astype('Int64')
games['user_score'] = pd.to_numeric(games['user_score'], errors='coerce')

2424

In [182]:
games.info()
games

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16713 entries, 0 to 16714
Data columns (total 11 columns):
name               16713 non-null object
platform           16713 non-null object
year_of_release    16444 non-null Int64
genre              16713 non-null object
na_sales           16713 non-null float64
eu_sales           16713 non-null float64
jp_sales           16713 non-null float64
other_sales        16713 non-null float64
critic_score       8137 non-null Int64
user_score         7590 non-null float64
rating             9949 non-null object
dtypes: Int64(2), float64(5), object(4)
memory usage: 1.6+ MB


,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,Wii Sports,Wii,2006,sports,41.36,28.96,3.77,8.45,76,8.0,E
1,Super Mario Bros.,NES,1985,platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008,racing,15.68,12.76,3.79,3.29,82,8.3,E
3,Wii Sports Resort,Wii,2009,sports,15.61,10.93,3.28,2.95,80,8.0,E
4,Pokemon Red/Pokemon Blue,GB,1996,role-playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
16710,Samurai Warriors: Sanada Maru,PS3,2016,action,0.00,0.00,0.01,0.00,NaN,NaN,NaN
16711,LMA Manager 2007,X360,2006,sports,0.00,0.01,0.00,0.00,NaN,NaN,NaN
16712,Haitaka no Psychedelica,PSV,2016,adventure,0.00,0.00,0.01,0.00,NaN,NaN,NaN
16713,Spirits & Spells,GBA,2003,platform,0.01,0.00,0.00,0.00,NaN,NaN,NaN


In [183]:
games[games['critic_score'].isna()]['year_of_release'].describe()

count    8461.000000
mean     2005.819643
std         7.044294
min      1980.000000
25%      2001.000000
50%      2008.000000
75%      2011.000000
max      2016.000000
Name: year_of_release, dtype: float64

In [184]:
games[games['rating'].isna()]['year_of_release'].describe()

count    6676.000000
mean     2004.883463
std         7.491332
min      1980.000000
25%      1999.000000
50%      2007.000000
75%      2011.000000
max      2016.000000
Name: year_of_release, dtype: float64

In [185]:
def total_sales(row):
    return row['na_sales'] +row['eu_sales'] + row['jp_sales'] + row['other_sales']

games['total_sales'] = games.apply(total_sales, axis=1)
games.head(10)

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,total_sales
0,Wii Sports,Wii,2006,sports,41.36,28.96,3.77,8.45,76,8.0,E,82.54
1,Super Mario Bros.,NES,1985,platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN,40.24
2,Mario Kart Wii,Wii,2008,racing,15.68,12.76,3.79,3.29,82,8.3,E,35.52
3,Wii Sports Resort,Wii,2009,sports,15.61,10.93,3.28,2.95,80,8.0,E,32.77
4,Pokemon Red/Pokemon Blue,GB,1996,role-playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN,31.38
5,Tetris,GB,1989,puzzle,23.20,2.26,4.22,0.58,NaN,NaN,NaN,30.26
6,New Super Mario Bros.,DS,2006,platform,11.28,9.14,6.50,2.88,89,8.5,E,29.80
7,Wii Play,Wii,2006,misc,13.96,9.18,2.93,2.84,58,6.6,E,28.91
8,New Super Mario Bros. Wii,Wii,2009,platform,14.44,6.94,4.70,2.24,87,8.4,E,28.32
9,Duck Hunt,NES,1984,shooter,26.93,0.63,0.28,0.47,NaN,NaN,NaN,28.31


In [186]:
games.query('year_of_release < 1994')['rating'].value_counts()

T       2
E10+    1
M       1
Name: rating, dtype: int64

### Вывод
Пропуски в оценках пользователя, критиков и в рейтинге не были заменены, так как это важные показатели для нашего анализа. Данные могли быть пропущены из-за небольшого количества оценок. Пропуски в рейтинге ESRB в старых играх скорее всего из-за того, что система ESRB появилась только в 1994. Пропуски в годах будут проигнорированы, скорей всего это была недостающая информация при парсинге данных или откуда брали данные о играх. Из названий некоторых игр можно "достать" год выпуска, но таких игр немного. В столбцах жанра и имени были 2 пропуска в одних и тех же записях - они были удалены из таблицы. 

Год и оценка критиков были приведены к целочисленному значению, оценка пользователей была приведена к формату с плавающей точкой, *'tbd'* были заменены на NaN.

Был добавлен столбец `total_sales` с суммой продаж по всему миру.

### 2. Исследовательский анализ данных.

In [187]:
numb_games_yearly = games.pivot_table(index='year_of_release', values='name', aggfunc='count').reset_index()
fig = px.line(numb_games_yearly, x='year_of_release', y='name', title='Количество выпущенных игр в год')
fig.show()



Начиная с 1994 года наблюдается рост количества выпускаемых игр, в нашем анализе не будем использовать информацию раньше этого года:

In [188]:
games_new = games.query('year_of_release >= 1994')

In [189]:
games_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16079 entries, 0 to 16714
Data columns (total 12 columns):
name               16079 non-null object
platform           16079 non-null object
year_of_release    16079 non-null Int64
genre              16079 non-null object
na_sales           16079 non-null float64
eu_sales           16079 non-null float64
jp_sales           16079 non-null float64
other_sales        16079 non-null float64
critic_score       7980 non-null Int64
user_score         7460 non-null float64
rating             9764 non-null object
total_sales        16079 non-null float64
dtypes: Int64(2), float64(6), object(4)
memory usage: 1.6+ MB


Найдем топ 10 лидирующих по продажам платформ:

In [190]:
top10_platforms = games_new.groupby('platform')['total_sales'].sum().sort_values(ascending=False)[:10]

In [191]:
display(top10_platforms.index) # список названий 10 топовых платформ
top10_df = games_new.query('platform in @top10_platforms.index')
top10_df

Index(['PS2', 'X360', 'PS3', 'Wii', 'DS', 'PS', 'PS4', 'GBA', 'PSP', '3DS'], dtype='object', name='platform')

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,total_sales
0,Wii Sports,Wii,2006,sports,41.36,28.96,3.77,8.45,76,8.0,E,82.54
2,Mario Kart Wii,Wii,2008,racing,15.68,12.76,3.79,3.29,82,8.3,E,35.52
3,Wii Sports Resort,Wii,2009,sports,15.61,10.93,3.28,2.95,80,8.0,E,32.77
6,New Super Mario Bros.,DS,2006,platform,11.28,9.14,6.50,2.88,89,8.5,E,29.80
7,Wii Play,Wii,2006,misc,13.96,9.18,2.93,2.84,58,6.6,E,28.91
...,...,...,...,...,...,...,...,...,...,...,...,...
16708,Woody Woodpecker in Crazy Castle 5,GBA,2002,platform,0.01,0.00,0.00,0.00,NaN,NaN,NaN,0.01
16709,SCORE International Baja 1000: The Official Game,PS2,2008,racing,0.00,0.00,0.00,0.00,NaN,NaN,NaN,0.00
16710,Samurai Warriors: Sanada Maru,PS3,2016,action,0.00,0.00,0.01,0.00,NaN,NaN,NaN,0.01
16711,LMA Manager 2007,X360,2006,sports,0.00,0.01,0.00,0.00,NaN,NaN,NaN,0.01


In [192]:
fig_data = top10_df.pivot_table(index=['platform', 'year_of_release'], values='total_sales', aggfunc='sum').reset_index()
fig = px.line(fig_data, x='year_of_release', y='total_sales', color='platform', title='Суммарные продажи топ-10 платформ')
fig.show()
box_fig = px.histogram(top10_df, x='year_of_release', y='total_sales', color='platform', barmode='overlay', marginal='box', title='Суммарные продажи топ-10 платформ')
box_fig.show()

In [193]:
data = top10_df.pivot_table(index=['platform'], values='year_of_release', aggfunc=['min', 'max']).reset_index()
data.columns=data.columns.droplevel(1)
data['lifetime'] = data['max'] - data['min']
last_year = max(data['max'])
avg_lifetime = round(data.query("max<@last_year")["lifetime"].mean())
# при подсчете средней продолжительности жизни платформы не будем учитывать те платформы, которые на момент 2016 года еще существуют
print(f'Средняя продолжительность жизни платформы {avg_lifetime:.0f} лет')


Средняя продолжительность жизни платформы 9 лет


In [194]:
data.sort_values(by='min', ascending=False)

,platform,min,max,lifetime
6,PS4,2013,2016,3
0,3DS,2011,2016,5
5,PS3,2006,2016,10
8,Wii,2006,2016,10
9,X360,2005,2016,11
1,DS,2004,2013,9
7,PSP,2004,2015,11
2,GBA,2000,2007,7
4,PS2,2000,2011,11
3,PS,1994,2003,9


Платформы, на которые еще выпускают игры, при приближении к 2016 году испытывают спад продаж, кроме PS4. Самые "молодые" платформы - PS4 и 3DS, они, по моему мнению и есть самые потенциально прибыльные платформы, они же и лидируют по глобальным продажам в 2014-2015 году.

In [195]:
fig = px.box(top10_df, y='total_sales', title='Суммарные продажи')
fig.show()


In [196]:
top10_df.query('total_sales==0')

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,total_sales
16676,G1 Jockey 4 2008,PS3,2008,sports,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0
16709,SCORE International Baja 1000: The Official Game,PS2,2008,racing,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0


Есть 2 игры с нулевыми продажами и отсутствующими оценками и рейтингом, избавимся от них:

In [197]:
top10_df.drop([16676, 16709], inplace=True)
games_new.drop([16676, 16709], inplace=True)

/Users/olga/anaconda3/envs/practicum/lib/python3.7/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [198]:
fig = px.box(top10_df, y='total_sales', color='platform', title='Суммарные продажи')
fig.show()

In [199]:
fig = px.box(top10_df, y='total_sales', color='platform', range_y=[0,2], title='Суммарные продажи')
fig.show()

In [200]:
compare_top10 = top10_df.pivot_table(index='platform', values='total_sales', aggfunc=['mean', 'median'])
compare_top10

,mean,median
,total_sales,total_sales
platform,,
3DS,0.503535,0.12
DS,0.378660,0.11
GBA,0.385795,0.16
PS,0.611412,0.26
PS2,0.580226,0.23
PS3,0.713670,0.28
PS4,0.801378,0.20
PSP,0.242691,0.09


Среди глобальных продаж топовых платформ очень много выбросов за пределами Q3, что смещает среднее значение, тогда как медианы все находятся в районе 0.1-0.2 млн. От больших значений продаж не будем избавляться, нам же хочется узнать, что влияет не невероятный успех игры.

Посмотрим, как влияют на продажи внутри одной популярной платформы отзывы пользователей и критиков. Посчитаем корреляцию между отзывами и продажами:

In [201]:
most_pop_platform = top10_df.dropna()
most_pop_platform = most_pop_platform.query('platform == "Wii"')

fig = px.scatter(most_pop_platform, x = 'total_sales', y = 'user_score', title = 'Зависимость глобальных продаж от пользовательских оценок')
fig.show()

fig = px.scatter(most_pop_platform, x = 'total_sales', y = 'critic_score', title = 'Зависимость глобальных продаж от оценок критиков')
fig.show()


In [202]:
def check_corr(platform):
    check_corr = top10_df.query('platform == @platform')[['total_sales', 'critic_score', 'user_score']]
    print(f'\nМатрица корреляции для {platform}')
    return check_corr.corr()
    
for platform in top10_df['platform'].unique():
    print(check_corr(platform))


Матрица корреляции для Wii
              total_sales  critic_score  user_score
total_sales      1.000000      0.177907    0.112831
critic_score     0.177907      1.000000    0.688285
user_score       0.112831      0.688285    1.000000

Матрица корреляции для DS
              total_sales  critic_score  user_score
total_sales      1.000000      0.237546    0.130876
critic_score     0.237546      1.000000    0.653908
user_score       0.130876      0.653908    1.000000

Матрица корреляции для X360
              total_sales  critic_score  user_score
total_sales      1.000000      0.392866    0.114187
critic_score     0.392866      1.000000    0.625020
user_score       0.114187      0.625020    1.000000

Матрица корреляции для PS3
              total_sales  critic_score  user_score
total_sales      1.000000      0.432759    0.128416
critic_score     0.432759      1.000000    0.631154
user_score       0.128416      0.631154    1.000000

Матрица корреляции для PS2
              total_sales  c

Удивительно, но оценки критиков и пользователей не связаны с глобальными продажами, но связаны между собой. Скорее всего, велась хорошая рекламная компания, которая и повлияла на глобальные продажи.

Посмотрим на общее распределение игр по жанрам:

In [203]:
genre_games_count = games_new.pivot_table(index='genre', values='name', aggfunc='count').sort_values(by='name', ascending=False)
genre_games_count

,name
genre,
action,3226
sports,2259
misc,1707
role-playing,1451
adventure,1284
shooter,1258
racing,1210
simulation,849
platform,819


Больше всего выпускалось игр в жанре action и sports.

In [204]:
genre_games_sales = games_new.pivot_table(index='genre', values='total_sales', aggfunc='sum').sort_values(by='total_sales', ascending=False)
genre_games_sales

,total_sales
genre,
action,1646.62
sports,1266.74
shooter,976.24
role-playing,895.60
misc,776.53
racing,694.59
platform,640.59
fighting,411.05
simulation,381.84


Эти же жанры собрали больше всего продаж.

In [205]:
genre_games = genre_games_sales.merge(genre_games_count['name'], on = 'genre')


genre_games = genre_games.rename(columns={"name": "games_count"})

genre_games['sales_per_game'] = genre_games['total_sales'] / genre_games['games_count']
genre_games.sort_values(by='sales_per_game', ascending=False)

,total_sales,games_count,sales_per_game
genre,,,
platform,640.59,819,0.782161
shooter,976.24,1258,0.776025
role-playing,895.60,1451,0.617229
racing,694.59,1210,0.574041
sports,1266.74,2259,0.560753
action,1646.62,3226,0.510422
fighting,411.05,814,0.504975
misc,776.53,1707,0.454909
simulation,381.84,849,0.449753


Но если посмотреть на средние продажи на игру, то больше денег получили игры в жанре platform и shooter.

### Вывод

Актуальным периодом будем считать 1994-2016 года. Рейтинг платформ по глобальным продажам: 'PS2', 'X360', 'PS3', 'Wii', 'DS', 'PS', 'PS4', 'GBA', 'PSP', '3DS', но так как средняя продолжительность жизни платформы 9 лет - потенциально прибыльными и жизнеспособными могут быть PS4 и 3DS, они лидируют по глобальным продажам в 2014-2015 году.

Среди глобальных продаж топовых платформ очень много выбросов за пределами Q3, что смещает среднее значение, тогда как медианы все находятся в районе 0.1-0.2 млн. 

Удивительно, но оценки критиков и пользователей не связаны с глобальными продажами, но связаны между собой. Скорее всего, велась хорошая рекламная компания, которая и повлияла на глобальные продажи.

Больше всего выпускалось игр в жанре action и sports. Эти же жанры собрали больше всего продаж.
Но если посмотреть на средние продажи на игру, то больше денег получили игры в жанре platform и shooter. Скорее всего в этих жанрах есть несколько очень популярных игр.

### 3. Портрет пользователя каждого региона.


Определим для пользователя каждого региона (NA, EU, JP):
- Самые популярные платформы (топ-5). 
- Самые популярные жанры (топ-5). 
- Влияет ли рейтинг ESRB на продажи в отдельном регионе?

In [206]:
region_top_platform = games_new.pivot_table(index = 'platform', values=['na_sales','eu_sales', 'jp_sales'], aggfunc='sum').reset_index()
display(region_top_platform.sort_values(by='eu_sales', ascending=False)[['platform','eu_sales']][:5])
display(region_top_platform.sort_values(by='jp_sales', ascending=False)[['platform','jp_sales']][:5])
region_top_platform.sort_values(by='na_sales', ascending=False)[['platform','na_sales']][:5]


,platform,eu_sales
14,PS2,332.63
15,PS3,327.21
26,X360,268.32
24,Wii,258.32
13,PS,212.39


,platform,jp_sales
3,DS,175.00
13,PS,139.78
14,PS2,137.54
1,3DS,100.62
15,PS3,79.41


,platform,na_sales
26,X360,595.74
14,PS2,572.92
24,Wii,486.87
15,PS3,390.13
3,DS,380.31


В Европе с небольшим отрывом не первом месте PS2 и P3 на втором, следом идет X360, Wii и еще более старый PS. DS на первом месте по продажам в Японии, а также он есть в топ-5 платформ Америки, скорей всего потому что в Европе эта платформа выпускалась уже только после Европы и США. В Японии популярен PS и его более новые версии. В Америке на первом месте X360, потом PS2 и их главный конкурент - Wii, следом - набирающая популярность PS3, и на 5 месте - DS. Microsoft со своим X360 не смог потеснить Sony и Nintendo в Японии, и их перестали там продавать

In [207]:
region_top_genre = games_new.pivot_table(index = 'genre', values=['na_sales','eu_sales', 'jp_sales'], aggfunc='sum').reset_index()
display(region_top_genre.sort_values(by='eu_sales', ascending=False)[['genre','eu_sales']][:5])
display(region_top_genre.sort_values(by='jp_sales', ascending=False)[['genre','jp_sales']][:5])
region_top_genre.sort_values(by='na_sales', ascending=False)[['genre','na_sales']][:5]

,genre,eu_sales
0,action,503.47
10,sports,367.90
8,shooter,310.80
6,racing,230.32
3,misc,209.69


,genre,jp_sales
7,role-playing,321.40
0,action,147.30
10,sports,109.54
3,misc,99.97
4,platform,84.07


,genre,na_sales
0,action,814.68
10,sports,657.57
8,shooter,531.30
3,misc,393.92
6,racing,342.82



Жанровые вкусы в Европе и Америке практически одинаковы, а вот в Японии на первом месте с большим отрывом ролевые игры, и еще японцы предпочитают платформеры гонкам.

In [208]:
games_new_pivot = games_new.pivot_table(index='rating', values=['na_sales','eu_sales', 'jp_sales'], aggfunc='sum').reset_index() 
fig_na = px.bar(games_new_pivot, x='rating', y='na_sales', title='Зависимость продаж в США от рейтинга ESRB')
fig_na.show()

fig_eu = px.bar(games_new_pivot, x='rating', y='eu_sales', title='Зависимость продаж в Европе от рейтинга ESRB')
fig_eu.show()

fig_jp = px.bar(games_new_pivot, x='rating', y='jp_sales', title='Зависимость продаж в Японии от рейтинга ESRB')
fig_jp.show()

Во всех регионах максимальные продажи у игр с рейтингом 'E' (для всех), затем идут игры для подростков ('T'), для тех кому больше 17 ('M'), для тех кому больше 10 ('E10+').

### Вывод
Вкусы пользователей в Японии отличаются от Европы и США, топ 5 платформ и жанров отличаются между ними. Рейтинг ESRB 'для всех' собирает больше всего продаж по всему миру.

### 4. Проверка гипотез.

H0: средние пользовательские рейтинги платформ Xbox One и PC одинаковые;  
H1: средние пользовательские рейтинги платформ Xbox One и PC различаются;

In [210]:
xbox = games_new.query('platform=="X360"')['user_score'].dropna()
PC = games_new.query('platform=="PC"')['user_score'].dropna()
print(f'Средние пользовательские оценки для X360 - {xbox.mean(): .2f} и PC - {PC.mean(): .2f}')

# проверим дисперсии выборок рейтингов для двух платформ:
xbox_var = np.var(xbox)
PC_var = np.var(PC) 
print(f'Разница между дисперсиями: {(xbox_var-PC_var)/PC_var: .2f}')  

Средние пользовательские оценки для X360 -  6.68 и PC -  7.07
Разница между дисперсиями: -0.10


In [211]:
alpha = 0.05 # пороговое значение статистической значимости
results = st.ttest_ind(
    xbox,
    PC,
    equal_var=False)

print('p-value:', results.pvalue)
if results.pvalue < alpha:
    print('Отвергаем H0')
else:
    print('Не получилось отвергнуть H0')

p-value: 8.351121816835871e-08
Отвергаем H0


Вероятность, что данные о различиях в пользовательских рейтингах для xbox и PC случайны крайне мала. Различия в оценках между платформами можем считать статистически значимыми.

---


H0: Средние пользовательские рейтинги жанров Action и Sports одинаковые;  
H1: Средние пользовательские рейтинги жанров Action и Sports разные;

In [ ]:
action = games_new.query('genre=="action"')['user_score'].dropna()
sports = games_new.query('genre=="sports"')['user_score'].dropna()
print(f'Средние пользовательские оценки для action - {action.mean(): .2f} и sports - {sports.mean(): .2f}')

# проверим дисперсии выборок рейтингов для двух платформ:
action_var = np.var(action)
sports_var = np.var(sports) 
print(f'Разница между дисперсиями: {(action_var-sports_var)/sports_var: .2f}')  

In [ ]:
alpha = 0.05 # пороговое значение статистической значимости
results = st.ttest_ind(
    action,
    sports,
    equal_var=False)

print('p-value:', results.pvalue)
if results.pvalue < alpha:
    print('Отвергаем H0')
else:
    print('Не получилось отвергнуть H0')

Вероятность, что данные о различиях в пользовательских рейтингах жанра action и sports случайна, 7%. Различия в средних оценках не можем считать статистически значимыми.

## 5. Общий вывод.

Были обработаны исторические данные о компьютерных играх. Данные хорошего качества, были только значительны пропуски в оценках пользователя, критиков и в рейтинге ESRB. Они не были заменены или удалены, так как это важные показатели для нашего анализа. 

Вкусы пользователей в Японии отличаются от Европы и США, топ 5 платформ и жанров отличаются между ними. Рейтинг ESRB 'для всех' собирает больше всего продаж по всему миру.

Для успешности игры нужно, чтоб она была для всех, т.е. с рейтингом ESRB 'E', жанр action или sports на PS4 или 3DS платформах.